# Welcome to the Agentic part of this guide
## Before we start discussing, let's get a recap of what we did last time: -
1. We learnt what Agents are, where they are used and difference between them and normal generative AI.
2. We learnt what APIs are and coded OpenAI's API in *python* to access ChatGPT.

- Now, if we put our logic in coding and theory we discussed last time: -
- We find out that Agents are basically automated AIs, now before coding, let's take a look at an example that doesn't require coding
- Go to this website called [n8n](https://n8n.io/), where we can build agents in a diagramtic/flowchart format.
- This website requires a subscription, but you have limited tries to build an agent.
- Try it for your self, put your logic in and learn how automation works, where if one task is completed, many other tasks start.

## **Automation** is when tasks are done by machines or software without needing people to do them step by step.
- Example: Instead of you sending an email to *100 people one by one*, a *script* or tool can send them all automatically.
- We can code automations in *python* and use them with the API we learnt, that creates an AI Agent, and your first step towards learning the future.

In [ ]:
# Imports
from openai import OpenAI
from pypdf import PdfReader
from dotenv import load_dotenv
import gradio as gr
import os

## Now, these imports are necessary for AI agents to do tasks for you, agentic tasks in an automated manner: -
1. `openai` - access to openai api
2. `pypdf` - from it we import `PdfReader` for access to reading pdfs
3. `dotenv` - loading environment variables
4. `gradio` - giving this app a GUI (Graphical User Interface)
5. `os` - for accessing files on your system

In [ ]:
load_dotenv(override=True)
api = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=api)

In [ ]:
reader = PdfReader("samples/sample1.pdf")
chapter = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        chapter += text

- `reader = PdfReader("samples/sample1.pdf")` - load PDF file into reader object
- `chapter = ""` - create an empty string to store extracted text
- `for page in reader.pages:` - iterates over all pages in PDF
- `text = page.extract_text()` - gets text content of the current page
- `if text: chapter_name += text` - adds the page text to `chapter` if text exists
### After loop ends, the entire text of the pdf is intialized into the `chapter` variable

In [ ]:
print(chapter)

In [ ]:
with open("samples/sample1.pdf", "rb", encoding="utf-8") as f:
    summary = f.read()
name = "Structured Query Language (SQL)"

- `with open("samples/sample1.pdf", "rb", encoding="utf-8") as f` 
  - opens the pdf file in read binary (rb) mode, no need to know about read binary mode right now.
  - Uses utf-8 encoding.
  - `with` ensures file is closed after use.
  - `f` is file object, stored in it.
- `summary = f.read( )` - reads the entire content of the file and stores it in variable summary.
- `name = "Structured Query Language"` - variable assigned for the file's topic and context.

In [ ]:
system_prompt=f"""You are every student's favorite teacher who specializes in teaching grade 12 in India, teaching them chapter - {name}. \n
                  You are answering questions on {name}'s topic, particularly questions related to {name}'s syntax and content of the chapter. \n
                  Your responsibility is to represent a teacher, who is good in programming as well as teaching students. But, give feedback to 
                  students as faithfully as possible. Be helpful and engaging, as if talking to a future topper or a potiential breakthrough maker
                  in the tech industry. If you don't know the answer say so."""
system_prompt += f"\n\n## Summary:\n{summary}\n\n## Chapter\n{chapter}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as that favorite teacher."

- What we are doing is making some really good prompts that help that AI in really understanding the context of the questions and how they have to be answered.
- We are assigning those really good prompts into variable `system_prompt`, and adding information to it by doing `+=` to the `system_prompt` variable.

In [ ]:
system_prompt #intialize the final content of the variable
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages
    )
    return response.choices[0].message.content

- What we are doing is now intializing the OpenAI API for access to ChatGPT's `gpt-4o-mini` model, if open this and learn - [API_Guide](1_lab_startHere.ipynb), for the better of you.
- We are giving the the `"role": "system"` for custom instructions which we assigned to earlier `system_prompt` variable, which is a good practice for any AI assistant, whether it is ChatGPT, Gemini, Grok, DeepSeek; anything needs instructions for better understanding of context.

In [ ]:
from pydantic import BaseModel
class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str
eval_sys_prompt = f"""You are an evaluator that decides whether a response to a question is acceptable. \n
                      You are provided with a conversation between a User and an Agent. Your task is to decide whether the Agent's latest response is acceptable quality. \n
                      The Agent is playing the role of a teacher who is every student's favorite and is answering questions based on the topic {name} on a website. \n
                      The Agent has been instructed to be helpful and engaging, as if talking to a future topper or a potiential breakthrough maker in the tech industry. \n
                      The Agent has been provided with context on the chapter {name} of class XII in form of a PDF."""
eval_sys_prompt += f"\n\n##Summary:\n{summary}\n\n## Chapter:\n{chapter}\n\n"
eval_sys_prompt += f"With this context, please evaluate the latest response, replying whether the solution is acceptable and your feedback."

- We have used the `pydantic` library and imported the `BaseModel`.
- Then we have defined a class `Evaluation` and given it the object, the model that we are using for evaluation, that we imported.
- Fields of `class`: -
  - `is_acceptable: bool` - Only accepts boolean values, which are either true or false.
  - `feedback: str` - Only accepts the feedback as a string text.
- `eval_sys_prompts` - given for greater context on how evaluation should work and reply be judged.

In [ ]:
def evaluator_user_prompt(reply, message, history):
    user_prompt = f"Here's the conversation between the User and the Agent: \n\n{history}\n\n"
    user_prompt += f"Here's the latest message from the User: \n\n{message}\n\n"
    user_prompt += f"Here's the latest response from the Agent: \n\n{reply}\n\n"
    user_prompt += f"Please evaluate the response, replying with whether it is acceptable and your feedback."
    return user_prompt

- The above provides the information the evaluator AI needs.
- The information is then grouped into variables, `+=` adds info/data to the variable

In [ ]:
gemini = OpenAI(
    api_key = os.getenv("GOOGLE_API_KEY"),
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)
# This uses google's AI model gemini for evaluation

In [ ]:
def evaluate(reply, message, history) -> Evaluation:
    messages = [{"role": "system", "content": eval_sys_prompt}] + [{"role": "user", "content": evaluator_user_prompt(reply, message, history)}]
    response = gemini.beta.chat.completions.parse(model="gemini-2.0-flash", messages=messages, response_format=Evaluation)
    return response.choices[0].message.parsed

- We have used gemini's model - `gemini-2.0-flash`, giving it the response format that we made earlier and stored as `class Evaluation(BaseModel)`

In [ ]:
messages = [{"role": "system", "content": system_prompt}] + [{"role": "user", "content": "What is the full syntax of SQL?"}]
response = client.chat.completions.create(model="gpt-4o-mini", messages=messages)
reply = response.choices[0].message.content
reply

- We have again used `gpt-4o-mini` of OpenAI for the question to be sent to the model and response to be made.

In [ ]:
evaluate(reply, "What is the full syntax of SQL?", messages[:1])

In [ ]:
def rerun(reply, message, history, feedback):
    updated_system_prompt = system_prompt + "\n\n## Previous answer rejected\nYour just tried to reply, but the quality control rejected your reply\n"
    updated_system_prompt += f"## Your attempted answer:\n{reply}\n\n"
    updated_system_prompt += f"## Reason for rejection:\n{feedback}\n\n"
    messages = [{"role": "system", "content": updated_system_prompt}] + history + [{"role": "user", "content": message}]
    response = client.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

- We have now made a function that if something about the reply is not accepted, it will show to the AI what was wrong stored in the `updated_system_prompt` variable

In [ ]:
def chat(message, history):
    if "syntax" in message:
        system = system_prompt + "\n\nEverything in your reply needs to be student friendly - \
                                  it is mandatory that you respond only and entirely in a student-friendly language."
    else:
        system = system_prompt
    messages = [{"role": "system", "content": system}] + history + [{"role": "user", "content": message}]
    response = client.chat.completions.create(model="gpt-4o-mini", messages=messages)
    reply = response.choices[0].message.content

    evaluation = evaluate(reply, message, history)
    
    if evaluation.is_acceptable:
        print("Passed evaluation - returning reply")
    else:
        print("Failed evaluation - retrying")
        print(evaluation.feedback)
        reply = rerun(reply, message, history, evaluation.feedback)       
    return reply

- We have created a function to handle replies.
- With a criterion that if `"syntax"` is in the question statment asked by the student, then change the system_prompt for model to reply to.
- Then we have added an `if` statement, that if `evaluation` is acceptable - then print the output that the reply has passed.
- Or else, the output will be print that the reply has failed with corresponding feedback with the `rerun( )` function taking place if this happens.

In [ ]:
gr.ChatInterface(chat)

- We have made the interface in the above line using gradio

# Congrats, you have now just made a PDF analyst AI agent, which can answer any questions on SQL, the chapter of grade 12 NCERT Computer Science (book maker in India).
- Based upon the code I give you a situation.
<br>
<table>
    <tr>
        <td>
            <img src="2_multiTasks.png" style="display: block;">
        </td>
        <td>
            <h1>Smart PDF Q-A Challenge</h1>
            <p>Goal: Build a backend flow that reads a PDF, answers a user question using the OpenAI API, then evaluates that answer using the Gemini evaluation endpoint.</p>
            <h2>Overview</h2>
            <ul>
                <li>Use PyPDF to extract text from the PDF.
                <li>Send the extracted text (or a relevant excerpt) + user question to OpenAI API to generate a student-friendly answer.
                <li>Send the OpenAI answer, original question, and a short source excerpt to the Gemini evaluation endpoint (via its URL) to get a correctness/coverage critique.
                <li>Return both the answer and Gemini’s evaluation (printed output).
            </ul>
        </td>
    </tr>
</table>